In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [12]:
message = pd.read_csv('./data/train.csv')
message = message.drop(['id'], axis=1)
message['date'] = pd.to_datetime(message['date'])

message

,date,store_nbr,family,sales,onpromotion
0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,2013-01-01,1,BABY CARE,0.000,0
2,2013-01-01,1,BEAUTY,0.000,0
3,2013-01-01,1,BEVERAGES,0.000,0
4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.133,0
3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [13]:
#collecting the holidays
holiday = pd.read_csv('./data/holidays_events.csv')
store = pd.read_csv('./data/stores.csv')
holiday['date'] = pd.to_datetime(holiday['date'])
message['on_holiday'] = 0

for i in range(len(holiday)):
    if ((holiday['type'][i] == 'Holiday') | (holiday['type'][i] == 'Transfer')):
        if(holiday['locale'][i] == 'National'):
            message.loc[message['date'] == holiday['date'][i], 'on_holiday'] = 1
        else:
            for j in list(store.loc[store['state'] == holiday['locale_name'][i]]['store_nbr']):
                message.loc[(message['date'] == holiday['date'][i]) 
                             &(message['store_nbr'] == j), 'on_holiday'] = 1

    if (holiday['type'][i] == 'Additional'):
        if(holiday['locale'][i] == 'National'):
            message.loc[message['date'] == holiday['date'][i], 'on_holiday'] = 2
        else:
            for j in list(store.loc[store['state'] == holiday['locale_name'][i]]['store_nbr']):
                message.loc[(message['date'] == holiday['date'][i]) 
                             &(message['store_nbr'] == j), 'on_holiday'] = 2

    if (holiday['type'][i] == 'Bridge'):
        if(holiday['locale'][i] == 'National'):
            message.loc[message['date'] == holiday['date'][i], 'on_holiday'] = 3
        else:
            for j in list(store.loc[store['state'] == holiday['locale_name'][i]]['store_nbr']):
                message.loc[(message['date'] == holiday['date'][i]) 
                             &(message['store_nbr'] == j), 'on_holiday'] = 3
    
    if (holiday['type'][i] == 'Event'):
        if(holiday['locale'][i] == 'National'):
            message.loc[message['date'] == holiday['date'][i], 'on_holiday'] = 4
        else:
            for j in list(store.loc[store['state'] == holiday['locale_name'][i]]['store_nbr']):
                message.loc[(message['date'] == holiday['date'][i]) 
                             &(message['store_nbr'] == j), 'on_holiday'] = 4

    if (holiday['type'][i] == 'Work Day'):
        if(holiday['locale'][i] == 'National'):
            message.loc[message['date'] == holiday['date'][i], 'on_holiday'] = 5
        else:
            for j in list(store.loc[store['state'] == holiday['locale_name'][i]]['store_nbr']):
                message.loc[(message['date'] == holiday['date'][i]) 
                             &(message['store_nbr'] == j), 'on_holiday'] = 5

message.head()


,date,store_nbr,family,sales,onpromotion,on_holiday
0,2013-01-01,1,AUTOMOTIVE,0.0,0,1
1,2013-01-01,1,BABY CARE,0.0,0,1
2,2013-01-01,1,BEAUTY,0.0,0,1
3,2013-01-01,1,BEVERAGES,0.0,0,1
4,2013-01-01,1,BOOKS,0.0,0,1


In [14]:
#collecting stores type and cluster's label
message['cluster'] = 0
for i in range(len(store)):
    message.loc[message['store_nbr'] == store['store_nbr'][i], 'type'] = store['type'][i]
    message.loc[message['store_nbr'] == store['store_nbr'][i], 'cluster'] = store['cluster'][i]
    
message.head()


,date,store_nbr,family,sales,onpromotion,on_holiday,cluster,type
0,2013-01-01,1,AUTOMOTIVE,0.0,0,1,13,D
1,2013-01-01,1,BABY CARE,0.0,0,1,13,D
2,2013-01-01,1,BEAUTY,0.0,0,1,13,D
3,2013-01-01,1,BEVERAGES,0.0,0,1,13,D
4,2013-01-01,1,BOOKS,0.0,0,1,13,D


In [15]:
oil_price = pd.read_csv('./data/oil.csv')
oil_price['date'] = pd.to_datetime(oil_price['date'])

#replacing null values with the mean of the previous and next value
oil_price['dcoilwtico'][0] = 93.14  #first value
oil_price['dcoilwtico'][1175] = oil_price['dcoilwtico'][1176] #2 consecutive null values

null_index = oil_price.loc[oil_price['dcoilwtico'].isnull()].index.tolist()
for i in null_index:
    oil_price['dcoilwtico'][i] = (oil_price['dcoilwtico'][i-1] + [oil_price['dcoilwtico'][i+1]])/2

#joining the oil price to the main dataframe
message['oil_price'] = 0.0
for i in range(len(oil_price)):
    message.loc[message['date'] == oil_price['date'][i], 'oil_price'] = oil_price['dcoilwtico'][i]

message.head()

,date,store_nbr,family,sales,onpromotion,on_holiday,cluster,type,oil_price
0,2013-01-01,1,AUTOMOTIVE,0.0,0,1,13,D,93.14
1,2013-01-01,1,BABY CARE,0.0,0,1,13,D,93.14
2,2013-01-01,1,BEAUTY,0.0,0,1,13,D,93.14
3,2013-01-01,1,BEVERAGES,0.0,0,1,13,D,93.14
4,2013-01-01,1,BOOKS,0.0,0,1,13,D,93.14


In [16]:
#extracting the number of transactions as a feature
transaction = pd.read_csv('./data/transactions.csv')
transaction['date'] = pd.to_datetime(transaction['date'])
message = pd.merge(message, transaction, on=['date', 'store_nbr'], how='left')

message.head()

,date,store_nbr,family,sales,onpromotion,on_holiday,cluster,type,oil_price,transactions
0,2013-01-01,1,AUTOMOTIVE,0.0,0,1,13,D,93.14,NaN
1,2013-01-01,1,BABY CARE,0.0,0,1,13,D,93.14,NaN
2,2013-01-01,1,BEAUTY,0.0,0,1,13,D,93.14,NaN
3,2013-01-01,1,BEVERAGES,0.0,0,1,13,D,93.14,NaN
4,2013-01-01,1,BOOKS,0.0,0,1,13,D,93.14,NaN


In [12]:
message.to_csv('./features.csv', index=False)